## GroupBy Reshaping Pivot Table

In [1]:
import pandas as pd

In [2]:
data = {
    'Employee': ['John', 'Anna', 'Peter', 'Linda', 'James', 'Laura', 'Michael'],
    'Department': ['HR', 'IT', 'IT', 'HR', 'IT', 'HR', 'Finance'],
    'Salary': [60000, 75000, 80000, 62000, 70000, 59000, 85000],
    'Experience': [5, 8, 6, 7, 4, 3, 10]
}

In [4]:
df=pd.DataFrame(data)

In [5]:
df

,Employee,Department,Salary,Experience
0,John,HR,60000,5
1,Anna,IT,75000,8
2,Peter,IT,80000,6
3,Linda,HR,62000,7
4,James,IT,70000,4
5,Laura,HR,59000,3
6,Michael,Finance,85000,10


In [7]:
agg = df.groupby('Department').agg(
    avg_salary=('Salary', 'mean'),
    max_salary=('Salary', 'max'),
    avg_experience=('Experience', 'mean')
)


In [8]:
agg

,avg_salary,max_salary,avg_experience
Department,,,
Finance,85000.000000,85000,10.0
HR,60333.333333,62000,5.0
IT,75000.000000,80000,6.0


In [10]:
filtered=agg[agg['avg_salary']>80000]

In [11]:
filtered

,avg_salary,max_salary,avg_experience
Department,,,
Finance,85000.0,85000,10.0


In [12]:
df

,Employee,Department,Salary,Experience
0,John,HR,60000,5
1,Anna,IT,75000,8
2,Peter,IT,80000,6
3,Linda,HR,62000,7
4,James,IT,70000,4
5,Laura,HR,59000,3
6,Michael,Finance,85000,10


## Reshaping Melt & Stack

In [13]:
data = {
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Math': [90, 85, 80],
    'Science': [88, 92, 85],
    'English': [95, 85, 89]
}


In [14]:
df=pd.DataFrame(data)
df

,Name,Math,Science,English
0,Alice,90,88,95
1,Bob,85,92,85
2,Charlie,80,85,89


In [15]:
melted=df.melt(id_vars='Name',var_name='subject', value_name='Grade')
melted

,Name,subject,Grade
0,Alice,Math,90
1,Bob,Math,85
2,Charlie,Math,80
3,Alice,Science,88
4,Bob,Science,92
5,Charlie,Science,85
6,Alice,English,95
7,Bob,English,85
8,Charlie,English,89


In [ ]:
# Stacking data

In [16]:
stacked=df.set_index('Name').stack()
stacked

Name            
Alice    Math       90
         Science    88
         English    95
Bob      Math       85
         Science    92
         English    85
Charlie  Math       80
         Science    85
         English    89
dtype: int64

In [17]:
# Unstacking data

In [18]:
unstacked=stacked.unstack()
unstacked

,Math,Science,English
Name,,,
Alice,90,88,95
Bob,85,92,85
Charlie,80,85,89


##  Pivot Tables

In [23]:
data = {
    'OrderID': [1, 2, 3, 4, 5, 6],
    'Product': ['Shoes', 'Shoes', 'Shirt', 'Shirt', 'Shirt', 'Shoes'],
    'Category': ['Footwear', 'Footwear', 'Apparel', 'Apparel', 'Apparel', 'Footwear'],
    'Quantity': [2, 3, 1, 5, 2, 4],
    'Price': [50, 50, 30, 30, 30, 50],
    'City': ['New York', 'Los Angeles', 'New York', 'New York', 'Los Angeles', 'New York']
}

In [24]:
df=pd.DataFrame(data)
df

,OrderID,Product,Category,Quantity,Price,City
0,1,Shoes,Footwear,2,50,New York
1,2,Shoes,Footwear,3,50,Los Angeles
2,3,Shirt,Apparel,1,30,New York
3,4,Shirt,Apparel,5,30,New York
4,5,Shirt,Apparel,2,30,Los Angeles
5,6,Shoes,Footwear,4,50,New York


In [25]:
# Adding total revenue column

In [28]:
df['TotalRevenue'] = df['Quantity'] * df['Price']

In [30]:
pivot_table=df.pivot_table(index='City', columns='Product', values='TotalRevenue', aggfunc={'TotalRevenue':['sum','mean']})

In [31]:
pivot_table

mean          sum      
Product     Shirt  Shoes Shirt Shoes
City                                
Los Angeles  60.0  150.0    60   150
New York     90.0  150.0   180   300

In [32]:
# Margins in Pivot Tables


In [35]:
pivot_tables_with_totals=df.pivot_table(index='City', columns='Product', values='TotalRevenue', aggfunc='sum', margins=True)

In [37]:
pivot_tables_with_totals

Product,Shirt,Shoes,All
City,,,
Los Angeles,60,150,210
New York,180,300,480
All,240,450,690


## Handling Missing Data in GroupBy, pivot and reshaping

In [38]:
data_with_nan = {
    'OrderID': [1, 2, 3, 4, 5, 6],
    'Product': ['Shoes', 'Shoes', 'Shirt', 'Shirt', 'Shirt', 'Shoes'],
    'Category': ['Footwear', 'Footwear', 'Apparel', 'Apparel', 'Apparel', 'Footwear'],
    'Quantity': [2, 3, None, 5, 2, None],
    'Price': [50, 50, 30, 30, None, 50],
    'City': ['New York', 'Los Angeles', 'New York', 'New York', 'Los Angeles', 'New York']
}

In [39]:
df_nan=pd.DataFrame(data_with_nan)

In [40]:
df_nan

,OrderID,Product,Category,Quantity,Price,City
0,1,Shoes,Footwear,2.0,50.0,New York
1,2,Shoes,Footwear,3.0,50.0,Los Angeles
2,3,Shirt,Apparel,NaN,30.0,New York
3,4,Shirt,Apparel,5.0,30.0,New York
4,5,Shirt,Apparel,2.0,NaN,Los Angeles
5,6,Shoes,Footwear,NaN,50.0,New York


In [41]:
# Handling missing values in groupby

In [43]:
grouped_with_nan=df_nan.groupby('City')['Price'].mean()

In [44]:
grouped_with_nan

City
Los Angeles    50.0
New York       40.0
Name: Price, dtype: float64